# Support Vector Classifier

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv(r"train.csv")

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data["satisfaction"] = le.fit_transform(data["satisfaction"])
data["Customer Type"] = le.fit_transform(data["Customer Type"])



data["Class"] = data["Class"].map({ "Eco":1,"Eco Plus":2,"Business":3})


"""data["JobType"] = data["JobType"].map({" ?":np.nan, " Never-worked":0,
                                       " Without-pay":1," Self-emp-not-inc":2,
                                       " Self-emp-inc":3," Private":4,
                                         "Local-gov":5," State-gov":6," Federal-gov":7," Without-pay":8})"""




Y = data.iloc[:,-1]
X = data[["Class", "Online boarding","Inflight entertainment"]]

from sklearn.impute import SimpleImputer
imputer  = SimpleImputer(missing_values = np.nan, strategy = "mean")
missing_data = X.values
missing_data = missing_data.reshape(len(missing_data),X.shape[1])
imputer = imputer.fit(missing_data)
X = imputer.transform(missing_data)
X1 = pd.DataFrame(X, columns = ["Class", "Online boarding","Inflight entertainment"])


from sklearn.model_selection import train_test_split


x_train, x_test,y_train,y_test = train_test_split(X1,Y,test_size=0.33, random_state=0)

x_train = x_train.sort_index().values
y_train = y_train.sort_index().values
x_test = x_test.sort_index().values
y_test = y_test.sort_index().values

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)

x_test = sc.transform(x_test)


In [3]:
import time
start = time.time()
from sklearn.svm import SVC
svc = SVC(C = 0.1,gamma = 10 )

svc.fit(x_train,y_train)

y_pred = svc.predict(x_test)

y_pred

end = time.time()

print("SVM çalışma süresi", end - start)


SVM çalışma süresi 227.42781734466553


In [4]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)

cm

array([[16999,  2422],
       [ 3015, 11853]], dtype=int64)

In [5]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average = 'macro')))
    print("Recall Score: {}".format(recall_score(labels, preds, average = 'macro')))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average = 'macro')))
    
print_metrics(y_test, y_pred) 

Precision Score: 0.8398441003731714
Recall Score: 0.8362525656496493
Accuracy Score: 0.8414360290472163
F1 Score: 0.837782370350344


Destek vektör makinelerinde kullanılan 'C' ve '𝞼' hiperparametreleri

In [7]:
import time

start = time.time()
from sklearn.model_selection import GridSearchCV, cross_val_score

param_grid = {'gamma': [0.0001, 0.01, 0.1, 1, 10], "C" : [0.01, 1.0,100.0 ]}


grid = GridSearchCV(SVC(), param_grid=param_grid)
grid.fit(x_train, y_train)
print("Grid-Search with accuracy")
print("Best parameters:", grid.best_params_)
print("Best cross-validation score (accuracy)):", grid.best_score_)

end= time.time()
print("GridSearch SVC hesaplama zamanı", end-start)


Grid-Search with accuracy
Best parameters: {'C': 1.0, 'gamma': 1}
Best cross-validation score (accuracy)): 0.84358256122962
GridSearch SVC hesaplama zamanı 54777.46046257019


In [5]:
import optuna
from sklearn.model_selection import GridSearchCV, cross_val_score
from optuna import Trial, visualization

start = time.time()



def tune(objective):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params


def svc_objective(trial):  
    
    # gamma = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
    # C = [1, 10, 100, 1e3, 1e4, 1e5]
    
    c_ = trial.suggest_categorical("c", [0.01, 1.0,100.0 ])
    gamma_ = trial.suggest_categorical("gamma", [0.1, 1.0,10.0 ])
    svc = SVC(C = c_, gamma = gamma_)
    svc.fit(x_train,y_train)
    return svc.score(x_test, y_test)
    
    


svc_params = tune(svc_objective)

print(svc_params)

end = time.time()


print("Optuna çalışma süresi " + str(end-start) + " sn")

[I 2021-09-19 22:07:49,025] A new study created in memory with name: no-name-c203b6f3-c099-48ff-9212-53031c0c0b58
[I 2021-09-19 22:12:59,480] Trial 0 finished with value: 0.8413777013036251 and parameters: {'c': 100.0, 'gamma': 1.0}. Best is trial 0 with value: 0.8413777013036251.
[I 2021-09-19 22:26:09,143] Trial 1 finished with value: 0.8417568316369681 and parameters: {'c': 100.0, 'gamma': 0.1}. Best is trial 1 with value: 0.8417568316369681.
[I 2021-09-19 22:29:42,666] Trial 2 finished with value: 0.8413777013036251 and parameters: {'c': 100.0, 'gamma': 10.0}. Best is trial 1 with value: 0.8417568316369681.
[I 2021-09-19 22:46:17,356] Trial 3 finished with value: 0.8417568316369681 and parameters: {'c': 100.0, 'gamma': 0.1}. Best is trial 1 with value: 0.8417568316369681.
[I 2021-09-19 22:51:29,688] Trial 4 finished with value: 0.8413777013036251 and parameters: {'c': 1.0, 'gamma': 1.0}. Best is trial 1 with value: 0.8417568316369681.
[I 2021-09-19 22:56:32,979] Trial 5 finished wi

Best score: 0.8417568316369681

Optimized parameters: {'c': 100.0, 'gamma': 0.1}

{'c': 100.0, 'gamma': 0.1}
Optuna çalışma süresi 12240.700247526169 sn
